## **Matrioska v2 - LLM Orchestration System with File-Based Architecture**
![Matrioska](https://live.staticflickr.com/8646/16075618524_6f3b5b199e_b.jpg)

-----

### 👤 Author: **Adriel D. S. Andrade**
- [LinkedIn](https://www.linkedin.com/in/adriel-domingues-de-souza-andrade/)

- [Github repo](https://github.com/adriel007/matrioska)

- [Google Colab](https://colab.research.google.com/drive/1Vq3b7Xu5z2Un0n3_6_dQVYWQrX4fsK0j)

### 📋 Overview

Matrioska v2 is an **advanced orchestration system for large language models (LLMs)** that implements a modular architecture based on files with shared state. Inspired by the concept of Russian nesting dolls, the system decomposes complex tasks into specialized files that communicate via a **shared whiteboard** (`shared_state`).

-----

### 🎯 Key Features

  * **📁 File-Based Architecture:** Automatic decomposition of projects into ordered files.
  * **🧠 Shared State:** Communication system between files via `shared_state`.
  * **💾 Full Persistence:** Checkpoints of architecture and state between executions.
  * **⚡ Sequential Generation:** Each file is generated in dependency order.
  * **🔗 Selective Context:** Files access only relevant information from predecessors.
  * **📦 Optimized Code:** Focus on minimal, complete, and efficient code using CDNs.

-----

### 🏗️ Architecture

#### Core Components

  * **`LocalLLM`** - Wrapper for Mistral models with 4-bit quantization.
  * **`MatrioskaOrchestrator`** - Main pipeline orchestrator.
  * **`ContextManager`** - Manages shared state and persistence.
  * **`Architecture`** - Data structure for file-based planning.
  * **`FileSpec`** - Individual file specification.
  * **`FileArtifact`** - Generated file artifact.

#### Execution Flow

$$
\begin{array}{ccc}
\text{PHASE 1: ARCHITECTURE} & \rightarrow & \text{PHASE 2: CODE GENERATION} \\
\downarrow & & \downarrow \\
\text{File Decomposition} & & \text{Sequential Generation} \\
& & \text{by Order/Dependency}
\end{array}
$$

-----

### 🚀 How to Use

#### Installation

```bash
pip install -q json-repair transformers accelerate bitsandbytes torch sentencepiece protobuf
```

#### Environment Cleanup (Optional)

```bash
!rm -rf /content/log
!rm -rf /content/matrioska_artifacts
!rm -rf /content/matrioska_checkpoints
```

#### Basic Execution

```python
from matrioska_v2 import LocalLLM, MatrioskaOrchestrator

# Initialize model
llm = LocalLLM("mistralai/Mistral-7B-Instruct-v0.3")
orchestrator = MatrioskaOrchestrator(llm, base_path="/content")

# Execute task
result = orchestrator.run("Create a library management system with authentication and dashboard")
```

#### Directory Structure

```
/content/
├── log/                        # Prompt and response logs
│   └── log.txt                # Complete generation history
├── matrioska_artifacts/        # Generated files
│   ├── index.html
│   ├── styles.css
│   └── app.js
└── matrioska_checkpoints/      # State and architecture
    ├── shared_state.json       # Shared whiteboard
    └── architecture.json       # Architectural plan
```

-----

### 📖 File System

#### File Specification (`FileSpec`)

```python
@dataclass
class FileSpec:
    name: str                          # Name without extension
    extension: str                     # File extension
    order: int                         # Creation order (1, 2, 3...)
    shared_state_writes: List[str]     # Info this file defines
    shared_state_reads: List[str]      # Info this file needs
    content: str                       # Code generation prompt
    details: str                       # Functional requirements
```

#### Architecture Example

```json
{
  "instructs": {
    "files": [
      {
        "name": "index",
        "extension": "html",
        "order": 1,
        "shared_state_writes": ["element_ids", "page_structure"],
        "shared_state_reads": [],
        "content": "Generate complete HTML structure for library system...",
        "details": "Responsive layout, login form, book catalog, dashboard"
      },
      {
        "name": "styles",
        "extension": "css",
        "order": 2,
        "shared_state_writes": ["css_classes", "color_scheme"],
        "shared_state_reads": ["element_ids", "page_structure"],
        "content": "Generate complete CSS using Tailwind CDN...",
        "details": "Modern design, dark mode, mobile-first"
      },
      {
        "name": "app",
        "extension": "js",
        "order": 3,
        "shared_state_writes": ["api_endpoints", "storage_keys"],
        "shared_state_reads": ["element_ids", "css_classes"],
        "content": "Generate JavaScript with authentication logic...",
        "details": "JWT auth, localStorage, CRUD operations"
      }
    ]
  }
}
```

#### Shared State Communication Example

```
# File 1 (HTML) generates IDs
SHARED_STATE_UPDATE:
{
  "element_ids": ["#loginForm", "#bookList", "#dashboardStats"],
  "page_structure": {
    "login": "section#login",
    "catalog": "section#catalog",
    "dashboard": "section#dashboard"
  }
}

# File 2 (CSS) automatically consumes IDs
# The ContextManager provides only the keys specified in shared_state_reads
```

-----

### 🔧 Model Configuration

#### 4-bit Quantization

```python
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)
```

#### Generation Parameters

  * `max_new_tokens`: 20,000 (configurable via `_MAX_TOKEN_`)
  * `temperature`: 0.3
  * `top_p`: 0.85
  * `do_sample`: True
  * `pad_token_id`: Auto (`eos_token_id`)

-----

### 📊 Architecture Prompt

The system uses **`ARCHITECT_SYSTEM_PROMPT`** which instructs the LLM to:

  * Decompose the task into independent files
  * Define creation order based on dependencies
  * Specify contracts via `shared_state_reads`/`writes`
  * Generate complete prompts for each file
  * Focus on minimal code and use of CDNs/libraries

#### Mandatory Prompt Rules

  * Strict JSON structure with `instructs` root
  * `order` field defining creation sequence
  * `shared_state_writes`: information the file defines
  * `shared_state_reads`: information the file needs
  * `content`: complete code generation prompt
  * `details`: functional and non-functional requirements

-----

### 💡 Use Cases

#### Complete Web System

```python
result = orchestrator.run('''
Create a complete e-commerce system with:
- Product catalog with search
- Shopping cart functionality
- User authentication
- Admin dashboard
- Responsive design with Tailwind CDN
''')
```

#### React/Vue Application

```python
result = orchestrator.run('''
Build a task management app using React CDN with:
- Component-based architecture
- State management
- CRUD operations
- LocalStorage persistence
''')
```

#### Data Dashboard

```python
result = orchestrator.run('''
Create an analytics dashboard with:
- Chart.js for visualizations
- Real-time data updates
- Export to CSV functionality
- Responsive grid layout
''')
```

-----

### 🎨 Output Example

```
================================================================================
🪆 MATRIOSKA ORCHESTRATOR - File-Based Architecture
================================================================================

🏗️  PHASE 1: ARCHITECTURE
--------------------------------------------------------------------------------
📋 Task: 'Create a library management system with authentication and dashboard'

✓ Project: Project_3_Files
✓ Files: 3
   1. index.html 📖[] ✍️['element_ids', 'page_structure']
   2. styles.css 📖['element_ids', 'page_structure'] ✍️['css_classes', 'color_scheme']
   3. app.js 📖['element_ids', 'css_classes'] ✍️['api_endpoints', 'storage_keys']

⚡ PHASE 2: CODE GENERATION
--------------------------------------------------------------------------------

🎯 Generating: index.html (Order: 1)
💾 index.html → /content/matrioska_artifacts/index.html
🧠 [SHARED STATE] Updated: ['element_ids', 'page_structure']
   ✍️ Wrote: ['element_ids', 'page_structure']
   ✓ Generated (2847 chars)

🎯 Generating: styles.css (Order: 2)
   📖 Reading context: ['element_ids', 'page_structure']
💾 styles.css → /content/matrioska_artifacts/styles.css
🧠 [SHARED STATE] Updated: ['css_classes', 'color_scheme']
   ✍️ Wrote: ['css_classes', 'color_scheme']
   ✓ Generated (1923 chars)

🎯 Generating: app.js (Order: 3)
   📖 Reading context: ['element_ids', 'css_classes']
💾 app.js → /content/matrioska_artifacts/app.js
🧠 [SHARED STATE] Updated: ['api_endpoints', 'storage_keys']
   ✍️ Wrote: ['api_endpoints', 'storage_keys']
   ✓ Generated (3456 chars)

✅ FINAL RESULT
================================================================================

📦 Project_3_Files

📂 Generated Files: 3
   1. index.html
   2. styles.css
   3. app.js

🧠 SharedState Keys: ['element_ids', 'page_structure', 'css_classes', 'color_scheme', 'api_endpoints', 'storage_keys']
================================================================================

📁 Artifacts: /content/matrioska_artifacts
🧠 SharedState: /content/matrioska_checkpoints/shared_state.json
```

-----

### 🔄 State Management

#### Shared State

  * **Persistent:** Saved in `shared_state.json` between executions.
  * **Structured:** JSON-serializable dictionary.
  * **Selective:** Files access only keys specified in `shared_state_reads`.
  * **Incremental:** Updated during the generation of each file.

#### Checkpoints

  * **Architecture:** `architecture.json` - Complete project plan
  * **SharedState:** `shared_state.json` - Current shared state
  * **Artifacts:** Individual files in `matrioska_artifacts/`
  * **Logs:** Complete history of prompts and responses in `log/log.txt`

#### Shared State Example (`shared_state.json`)

```json
{
  "element_ids": ["#loginForm", "#bookList", "#dashboard"],
  "page_structure": {
    "login": "section#login",
    "catalog": "section#catalog"
  },
  "css_classes": ["btn-primary", "card", "nav-item"],
  "color_scheme": {
    "primary": "#3b82f6",
    "secondary": "#8b5cf6"
  },
  "api_endpoints": {
    "login": "/api/auth/login",
    "books": "/api/books"
  },
  "storage_keys": ["authToken", "currentUser"]
}
```

#### 📦 SharedState Updates Extraction

The system automatically detects updates in the format:

```
// At the end of the generated code
SHARED_STATE_UPDATE:
{
  "key1": "value1",
  "key2": ["item1", "item2"]
}
```

This marker is:

  * Extracted and processed by the `ContextManager`
  * Removed from the final code
  * Persisted in `shared_state.json`

#### 📄 Returned API

```python
result = orchestrator.run("Create app...")

# Returns a dictionary with:
{
  "architecture": Architecture,     # Object with the project plan
  "artifacts": List[FileArtifact], # List of generated files
  "shared_state": Dict[str, Any]   # Final shared state
}
```

-----

### 🛠️ Technical Requirements

  * **GPU:** NVIDIA T4 (8GB VRAM) or superior
  * **RAM:** 12GB+ recommended
  * **Python:** 3.8+
  * **Libraries:**
      * `transformers` (Hugging Face)
      * `torch` (PyTorch)
      * `bitsandbytes` (Quantization)
      * `accelerate` (Optimization)
      * `json-repair` (Robust Parsing)
      * `sentencepiece`, `protobuf` (Tokenization)

-----

### 🔍 Logging and Debug

All prompts and responses are saved in `/content/log/log.txt`:

```
PROMPT:
==========================================
[Complete prompt sent to LLM]
==========================================
RESULT:
==========================================
[LLM Response]
```

-----

### 🎯 Best Practices

  * **File Order:** HTML/DB first → CSS/Styles → JS/Logic → API/Backend
  * **SharedState:** Define clear contracts between files (IDs, classes, routes)
  * **Detailed Prompts:** The `content` field must be a complete generation prompt
  * **CDNs:** Prioritize libraries via CDN to reduce complexity
  * **Minimal Code:** Focus on minimal and functional code

-----

### 🔮 Differences from v1

| Aspect | v1 (Modules) | v2 (Files) |
| :--- | :--- | :--- |
| **Basic Unit** | `ModuleSpec` | `FileSpec` |
| **Final Integration** | Artifact assembly | Independent files |
| **Structure** | 3 phases | 2 phases |
| **Focus** | Conceptual modularity | Practical code generation |
| **Output** | Integrated result | Separate files |

-----

### 📄 License

This project is intended for research and educational development purposes.

**Matrioska v2: Transforming ideas into structured code 🪆✨**

In [ ]:
!pip install -q json-repair transformers accelerate bitsandbytes torch sentencepiece protobuf # requirements.txt with specific versions in github repo
!cd /content/
!rm -rf /content/log
!rm -rf /content/matrioska_artifacts
!rm -rf /content/matrioska_checkpoints

import json
import os
import torch
from json_repair import repair_json
from dataclasses import dataclass, asdict, field
from typing import List, Optional, Dict, Any
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
_MAX_TOKEN_ = 20_000

ARCHITECT_SYSTEM_PROMPT = """You are Matrioska Architect. Decompose user requests into isolated modules that communicate via shared_state.

**OUTPUT FORMAT RULES (Mandatory):**

1.  **Structure:** The output **MUST** be a strict JSON object following the format below.
2.  **Root Object:** The root object **MUST** be named `instructs`.
3.  **Order (`order`):** Assign an integer starting from 1 to indicate the **creation order** of files (e.g., HTML/DB first, then CSS/JS/API logic).
4.  **Shared Information (`shared_state_reads/writes`):**
    * `shared_state_writes`: List **key information** this file defines (e.g., "HTML element IDs," "CSS class names," "API endpoint routes").
    * `shared_state_reads`: List **key information** this file requires from previous files in the `order` (e.g., CSS reads "HTML element IDs").
5.  **Content Requirement (`content`):** Must contain a **complete and detailed prompt** for a coding AI, demanding the generation of **full, reduced, and efficient code**. Explicitly instruct the use of **CDNs and lightweight libraries**.
6.  **Details (`details`):** Must contain a concise list of **functional and non-functional requirements**.

**Mandatory Output Structure:**

{
  "instructs": {
    "files": [
      {
        "name": "[FILE_NAME_WITHOUT_EXTENSION]",
        "extension": "[EXTENSION]",
        "order": 1,
        "shared_state_writes": ["[INFO_DEFINED_BY_THIS_FILE]"],
        "shared_state_reads": ["[INFO_NEEDED_FROM_OTHER_FILES]"],
        "content": "[DETAILED PROMPT FOR A CODING AI TO GENERATE THE ENTIRE CODE, FOCUSING ON BEING REDUCED, COMPLETE, AND USING CDNS/LIBRARIES IF NOT SPECIFIED]",
        "details": "[CONCISE FUNCTIONAL AND NON-FUNCTIONAL REQUIREMENTS]"
      }
    ]
  }
}

NOW PROCESS THIS REQUEST:"""


In [ ]:
class LocalLLM:
    def __init__(self, model_name: str = "mistralai/Mistral-7B-Instruct-v0.3"):
        print(f"🔄 Loading {model_name}...")

        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )

        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quant_config,
            device_map="auto",
            trust_remote_code=True
        )

        self.model_name = model_name
        print(f"✅ Model loaded: {model_name}")
        print(f"💾 VRAM used: {torch.cuda.memory_allocated()/1024**3:.2f}GB")

    def generate(self, prompt: str, max_tokens: int = _MAX_TOKEN_, system: str = "") -> str:
        if system:
            full_prompt = f"{system}\n\n{prompt}"
        else:
            full_prompt = prompt

        inputs = self.tokenizer(full_prompt, return_tensors="pt").to("cuda")

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=0.3,
                top_p=0.85,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )

        response = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:],
            skip_special_tokens=True
        )

        log_dir = "/content/log"
        log_file = os.path.join(log_dir, "log.txt")
        os.makedirs(log_dir, exist_ok=True)
        with open(log_file, "a") as f:
            f.write("PROMPT:\n==========================================\n\n")
            f.write(full_prompt)
            f.write("\n==========================================\n")
            f.write("RESULT:\n==========================================\n\n")
            f.write(response.strip())
            f.write("\n\n\n\n")

        return response.strip()


@dataclass
class FileSpec:
    name: str
    extension: str
    order: int
    shared_state_writes: List[str] = field(default_factory=list)
    shared_state_reads: List[str] = field(default_factory=list)
    content: str = ""
    details: str = ""


@dataclass
class Architecture:
    project_name: str
    files: List[FileSpec]


@dataclass
class FileArtifact:
    name: str
    extension: str
    order: int
    content: str
    shared_state_updates: Dict[str, Any] = field(default_factory=dict)


class ContextManager:
    def __init__(self, base_path: str = "/content"):
        self.base_path = base_path
        self.artifacts_dir = os.path.join(base_path, "matrioska_artifacts")
        self.checkpoints_dir = os.path.join(base_path, "matrioska_checkpoints")
        self.shared_state_file = os.path.join(self.checkpoints_dir, "shared_state.json")

        os.makedirs(self.artifacts_dir, exist_ok=True)
        os.makedirs(self.checkpoints_dir, exist_ok=True)

        self.shared_state: Dict[str, Any] = self._load_shared_state()

    def _load_shared_state(self) -> Dict[str, Any]:
        try:
            with open(self.shared_state_file, "r", encoding="utf-8") as f:
                state = json.load(f)
                print("🧠 [SHARED STATE] Context recovered")
                return state
        except FileNotFoundError:
            print("🧠 [SHARED STATE] Starting new context")
            return {}

    def _save_shared_state(self):
        with open(self.shared_state_file, "w", encoding="utf-8") as f:
            json.dump(self.shared_state, f, indent=2, ensure_ascii=False)

    def update_shared_state(self, updates: Dict[str, Any]):
        self.shared_state.update(updates)
        self._save_shared_state()
        print(f"🧠 [SHARED STATE] Updated: {list(updates.keys())}")

    def get_shared_context(self, keys: List[str]) -> Dict[str, Any]:
        context = {}
        for key in keys:
            if key in self.shared_state:
                context[key] = self.shared_state[key]
        return context

    def save_architecture(self, arch: Architecture):
        filepath = os.path.join(self.checkpoints_dir, "architecture.json")
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump({
                "project_name": arch.project_name,
                "files": [asdict(f) for f in arch.files]
            }, f, indent=2, ensure_ascii=False)
        print(f"💾 [CHECKPOINT] Architecture → {filepath}")

    def load_architecture(self) -> Optional[Architecture]:
        filepath = os.path.join(self.checkpoints_dir, "architecture.json")
        try:
            with open(filepath, "r", encoding="utf-8") as f:
                data = json.load(f)
            files = [FileSpec(**f) for f in data["files"]]
            print("📂 [RESTORATION] Context recovered")
            return Architecture(
                project_name=data["project_name"],
                files=files
            )
        except FileNotFoundError:
            return None

    def save_artifact(self, artifact: FileArtifact):
        filename = os.path.join(self.artifacts_dir, f"{artifact.name}.{artifact.extension}")
        with open(filename, "w", encoding="utf-8") as f:
            f.write(artifact.content)
        print(f"💾 {artifact.name}.{artifact.extension} → {filename}")

    def get_artifacts_path(self) -> str:
        return self.artifacts_dir


class MatrioskaOrchestrator:
    def __init__(self, llm: LocalLLM, base_path: str = "/content"):
        self.llm = llm
        self.context_manager = ContextManager(base_path)

    def run(self, task: str, verbose: bool = True):
        """Main pipeline with file-based architecture"""
        if verbose:
            print("\n" + "="*80)
            print("🪆 MATRIOSKA ORCHESTRATOR - File-Based Architecture")
            print("="*80 + "\n")

        if verbose:
            print("🏗️  PHASE 1: ARCHITECTURE")
            print("-" * 80)
        architecture = self._architecture_phase(task, verbose)

        if not architecture:
            return None

        self.context_manager.save_architecture(architecture)

        if verbose:
            print("\n⚡ PHASE 2: CODE GENERATION")
            print("-" * 80)
        artifacts = self._generation_phase(architecture, verbose)

        if verbose:
            print("\n✅ FINAL RESULT")
            print("=" * 80)
            self._display_results(architecture, artifacts)
            print("=" * 80)
            print(f"\n📁 Artifacts: {self.context_manager.get_artifacts_path()}")
            print(f"🧠 SharedState: {self.context_manager.shared_state_file}")

        return {
            "architecture": architecture,
            "artifacts": artifacts,
            "shared_state": self.context_manager.shared_state
        }

    def _architecture_phase(self, task: str, verbose: bool) -> Optional[Architecture]:
        if verbose:
            print(f"📋 Task: '{task}'")

        response = self.llm.generate(task, max_tokens=_MAX_TOKEN_, system=ARCHITECT_SYSTEM_PROMPT)

        try:
            data = json.loads(repair_json(response))

            if "instructs" not in data or "files" not in data["instructs"]:
                raise ValueError("Invalid response format: missing 'instructs.files'")

            files = [FileSpec(**f) for f in data["instructs"]["files"]]
            files.sort(key=lambda x: x.order)

            project_name = f"Project_{len(files)}_Files"
            arch = Architecture(project_name=project_name, files=files)

            if verbose:
                print(f"\n✓ Project: {arch.project_name}")
                print(f"✓ Files: {len(arch.files)}")
                for f in arch.files:
                    reads = f" 📖{f.shared_state_reads}" if f.shared_state_reads else ""
                    writes = f" ✍️{f.shared_state_writes}" if f.shared_state_writes else ""
                    print(f"   {f.order}. {f.name}.{f.extension}{reads}{writes}")

            return arch

        except Exception as e:
            if verbose:
                print(f"⚠️  Parsing failed: {e}")
                print(f"Response preview: {response[:500]}...")
            return None

    def _generation_phase(self, architecture: Architecture, verbose: bool) -> List[FileArtifact]:
        artifacts = []

        for file_spec in architecture.files:
            if verbose:
                print(f"\n🎯 Generating: {file_spec.name}.{file_spec.extension} (Order: {file_spec.order})")

            context = self.context_manager.get_shared_context(file_spec.shared_state_reads)

            if context and verbose:
                print(f"   📖 Reading context: {list(context.keys())}")

            context_text = ""
            if context:
                context_text = "\n\nAVAILABLE SHARED INFORMATION (from previous files):\n"
                for key, value in context.items():
                    context_text += f"- {key}: {json.dumps(value, ensure_ascii=False)}\n"

            prompt = f"""FILE: {file_spec.name}.{file_spec.extension}

GENERATION INSTRUCTIONS:
{file_spec.content}{context_text}

REQUIREMENTS:
{file_spec.details}

Generate the COMPLETE, REDUCED, and EFFICIENT code for this file. Use CDNs and lightweight libraries when appropriate.

If you define key information that other files need (e.g., element IDs, class names, API routes), list them at the end in the format:
SHARED_STATE_UPDATE:
{{
  "key1": "value1",
  "key2": ["item1", "item2"]
}}
"""

            content = self.llm.generate(prompt, max_tokens=_MAX_TOKEN_)

            updates = self._extract_shared_state_updates(content)

            if updates:
                self.context_manager.update_shared_state(updates)
                if verbose:
                    print(f"   ✍️ Wrote: {list(updates.keys())}")

            clean_content = self._remove_shared_state_marker(content)

            artifact = FileArtifact(
                name=file_spec.name,
                extension=file_spec.extension,
                order=file_spec.order,
                content=clean_content,
                shared_state_updates=updates
            )

            self.context_manager.save_artifact(artifact)
            artifacts.append(artifact)

            if verbose:
                print(f"   ✓ Generated ({len(clean_content)} chars)")

        return artifacts

    def _extract_shared_state_updates(self, content: str) -> Dict[str, Any]:
        try:
            marker = "SHARED_STATE_UPDATE:"
            if marker in content:
                after_marker = content[content.find(marker) + len(marker):]
                parsed = json.loads(repair_json(after_marker))

                if isinstance(parsed, list):
                    print(f"   ⚠️ SharedState as array, converting to dict")
                    return {"data_list": parsed}

                if not isinstance(parsed, dict):
                    print(f"   ⚠️ SharedState is invalid: {type(parsed).__name__}")
                    return {}

                return parsed

        except Exception as e:
            print(f"   ⚠️ Failed to extract shared_state: {e}")

        return {}

    def _remove_shared_state_marker(self, content: str) -> str:
        marker = "SHARED_STATE_UPDATE:"
        if marker in content:
            return content[:content.find(marker)].strip()
        return content

    def _display_results(self, arch: Architecture, artifacts: List[FileArtifact]):
        print(f"\n📦 {arch.project_name}")
        print(f"\n📂 Generated Files: {len(artifacts)}")
        for artifact in artifacts:
            print(f"   {artifact.order}. {artifact.name}.{artifact.extension}")
        print(f"\n🧠 SharedState Keys: {list(self.context_manager.shared_state.keys())}")

# **TODO LIST**
- [ ] Script for create correctly file extensions (extract from \`\`\` language `content`\`\`\`)
- [ ] Analysis/research prompt model (for reading->abstract tasks, using shared state to save key-words and means)

In [ ]:
!nvidia-smi

llm = LocalLLM("mistralai/Mistral-7B-Instruct-v0.3")
orchestrator = MatrioskaOrchestrator(llm, base_path="/content")

result = orchestrator.run(input("Enter your prompt: "))

import json
with open('/content/matrioska_checkpoints/shared_state.json', 'r') as f:
    shared_state = json.load(f)
print(json.dumps(shared_state, indent=2))

from google.colab import files
!zip -r matrioska_results.zip /content/matrioska_artifacts /content/matrioska_checkpoints
files.download('matrioska_results.zip')